# Лабораторная работа 6

Задачи:

* Провести классификацию русских текстов на несколько категорий. Лучше большой текст
* Провести предобработку текстов: нормализацию, лемматизацию и тд
* Сравнить эмбеддинги
* Попробовать несколько методов классификации (+косинусы)

Будем использовать Войну и мир, Краткую историю всего

In [1]:
# будем делить Войду и мир, Краткую историю всего на partsCount одинаковых частей
partsCount = 400
myTestSize = 0.2

textDir = "/home/alex/Downloads/bmstuML/texts/"

warFile = textDir + "voina_i_mir.txt"

istoriaFile = textDir + "kratkaya.txt"


with open(warFile, encoding="utf8") as f:
    war_text = f.read()
    
with open(istoriaFile, encoding="utf8") as f:
    istoria_text = f.read()

In [2]:
war_text[:300]

'\n\nЛев Толстой\n\nВойна и мир\n\nТома первый и второй\n\n\n\nВ.\xa0Шкловский «Война и мир» Льва Толстого\n\n\nЗамысел\n\nВ 1855 году появилось объявление об издании «Полярной звезды». На обложке книги в круге восходящего солнца были изображены пять портретов казненных декабристов; под портретами топор и подписано: «'

In [3]:
istoria_text[:300]

'\n«Краткая история почти всего на свете» Билла Брайсона — самая необычная энциклопедия из всех существующих! И это первая книга, которой была присуждена престижная европейская премия за вклад в развитие мировой науки имени Рене Декарта.\n\n\n\nСейчас, в начале 2003 года, я держу перед собой несколько стр'

### Токенизация

Выделим из текстов отдельные слова. Удалим ненужные "слова" (знаки препинания, лишние символы)


In [4]:
import nltk

nltk.download("punkt")
war_tokens = nltk.word_tokenize(war_text)

war_tokens[:15]

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Лев',
 'Толстой',
 'Война',
 'и',
 'мир',
 'Тома',
 'первый',
 'и',
 'второй',
 'В.',
 'Шкловский',
 '«',
 'Война',
 'и',
 'мир']

In [5]:
# второй текст
istoria_tokens = nltk.word_tokenize(istoria_text)

istoria_tokens[:15]

['«',
 'Краткая',
 'история',
 'почти',
 'всего',
 'на',
 'свете',
 '»',
 'Билла',
 'Брайсона',
 '—',
 'самая',
 'необычная',
 'энциклопедия',
 'из']

In [6]:
from snowballstemmer import RussianStemmer
stemmer = RussianStemmer()
stemmerResult = stemmer.stemWords(war_text.lower().split())

stemmerResult[:10]


['лев', 'толст', 'войн', 'и', 'мир', 'том', 'перв', 'и', 'втор', 'в.']

Как видим, слова получились "кривыми", простое обрезание окончания - не вариант. Воспользуемся лемматизатором.

In [7]:
import pymystem3

mystem = pymystem3.Mystem()

war_lemm = mystem.lemmatize(" ".join(war_tokens))

istoria_lemm = mystem.lemmatize(" ".join(istoria_tokens))



In [8]:
war_lemm[:10]

['лев', ' ', 'толстой', ' ', 'война', ' ', 'и', ' ', 'мир', ' ']

In [9]:
istoria_lemm[:10]

['« ', 'краткий', ' ', 'история', ' ', 'почти', ' ', 'все', ' ', 'на']

Посмотрим на морфологический анализ слов.

In [10]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
morph.parse("свет")

[Parse(word='свет', tag=OpencorporaTag('NOUN,inan,masc,Sgtm sing,nomn'), normal_form='свет', score=0.51923, methods_stack=((DictionaryAnalyzer(), 'свет', 557, 0),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,inan,masc,Sgtm sing,accs'), normal_form='свет', score=0.423076, methods_stack=((DictionaryAnalyzer(), 'свет', 557, 4),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,anim,femn,Name plur,gent'), normal_form='света', score=0.01923, methods_stack=((DictionaryAnalyzer(), 'свет', 237, 8),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,anim,femn,Name plur,accs'), normal_form='света', score=0.01923, methods_stack=((DictionaryAnalyzer(), 'свет', 237, 10),)),
 Parse(word='свет', tag=OpencorporaTag('NOUN,anim,femn,Name sing,voct,Infr'), normal_form='света', score=0.01923, methods_stack=((DictionaryAnalyzer(), 'свет', 237, 13),))]

In [11]:
# Удалим знаки, пустые слова, числа

bad_chars = ["«", "»",".",",","—",":","1","2","3","4","5","6","7","8","9","0","?","-","[","]", "стр", "(", ")",
            ";", "X", "!", "издание", "…", "’","&"]

def containsStr(a,b):
    for i in b:
        if i in a:
            return True
    return False

def deleteWords(wordList):
    resultList = [item for item in wordList if len(item) > 1 ]
    resultList = [item for item in resultList if not containsStr(item, bad_chars) ]
    return resultList



war_lemm_clear = deleteWords(war_lemm)

istoria_lemm_clear = deleteWords(istoria_lemm)


In [12]:
war_lemm_clear[:10]

['лев',
 'толстой',
 'война',
 'мир',
 'том',
 'первый',
 'второй',
 'шкловский',
 'война',
 'мир']

In [13]:
istoria_lemm_clear[:10]

['краткий',
 'история',
 'почти',
 'все',
 'на',
 'свет',
 'билл',
 'брайсон',
 'самый',
 'необычный']

In [14]:
war_text_clear = " ".join(war_lemm_clear)
istoria_text_clear = " ".join(istoria_lemm_clear)

In [15]:
war_text_clear[:300]

'лев толстой война мир том первый второй шкловский война мир лев толстой замысел год появляться объявление об полярный звезда на обложка книга круг восходить солнце быть изображать пять портрет казнить декабрист под портрет топор подписывать июль год том помечать день казнь декабрист над заглавие туч'

In [16]:
# сохраним текста

# with open(textDir+ '1.txt', 'w') as file:
#     file.write(war_text_clear)
    
# with open(textDir+ '2.txt', 'w') as file:
#     file.write(istoria_text_clear)  


In [17]:
# функция разделения на части
def chunkify(lst,n):
    return [lst[i::n] for i in range(n)]

In [18]:
from string import punctuation

# def lemmatize(input_text):
#     tokens = nltk.word_tokenize(input_text)
#     normed_tokens = [morph.parse(s)[0].normal_form for s in tokens]
#     # исключии стоп слова
#     normed_tokens = deleteWords(normed_tokens)
#     return ' '.join(normed_tokens)

In [19]:
import os
import pandas as pd
import numpy as np


# # nogroToByM uycTow naTtadpevim
# df = pd.DataFrame(columns=['text', 'class'])

# dir0 = textDir + "/0"
# dir1 = textDir + "/1"

# for filename in os.listdir(dir0) :
#     with open(os.path.join(dir0, filename), encoding='utf8') as file:
#         contents = lemmatize(file.read())

#     df = df.append(pd.Series({'text': contents, 'class': 0}), ignore_index=True)



In [20]:

# for filename in os.listdir(dir1) :
#     with open(os.path.join(dir1, filename), encoding='utf8') as file:
#         contents = lemmatize(file.read())

#     df = df.append(pd.Series({'text': contents, 'class': 1}), ignore_index=True)

In [21]:

war_parts = chunkify(war_lemm_clear, partsCount)
istoria_parts = chunkify(istoria_lemm_clear, partsCount)

istoria_parts[1][:10]

['история',
 'флэннерить',
 'точно',
 'яйцо',
 'же',
 'помогать',
 'содержать',
 'как',
 'шестидесятый',
 'ускоритель']

In [22]:
# объединяем слова в частях
# на выходе - список из текстов
def joinWordsToTextList(wordsLists):
    result = []
    for i in wordsLists:
        oneText = " ".join(i)
        result.append(oneText)
    return result


war_texts = joinWordsToTextList(war_parts)
istoria_texts = joinWordsToTextList(istoria_parts)

In [23]:
# war_texts[1][:300]

In [24]:
df = pd.DataFrame(columns=["text","class"])


for i in war_texts:
    df = df.append(pd.Series({"text":i, "class":0}), ignore_index=True)

# у второго типа текстов класс == 1
for i in istoria_texts:
    df = df.append(pd.Series({"text":i, "class":1}), ignore_index=True)
    

print("war_texts size:"+str(len(war_texts[3])), "\tistoria_text size:"+str(len(istoria_texts[3])))


war_texts size:3496 	istoria_text size:2585


In [25]:
df

,text,class
0,лев приобретать понимать состоять под как нет ...,0
1,толстой он сила только влияние бы мать борьба ...,0
2,война великий народ то исторический смерть тол...,0
3,мир слава слабость что событие она конец помощ...,0
4,том который декабрист мы война новый давать не...,0
...,...,...
795,рич куча отращивать добела предполагать малень...,1
796,из создавать плавник шар что что склоняться от...,1
797,музей живой конечность из мочь крошечный велич...,1
798,штат существо щеголять железо сам точка млрд б...,1


In [26]:
# разбив на трейн и тест
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["text"], df["class"], test_size=myTestSize, random_state=42)

In [27]:
X_train

264    хотеть год роман отражение андрей тот один сил...
615    естественный небольшой век этот все размышлени...
329    стиль быть история осуществляться взятка котор...
342    восторг причем думать выражать как становиться...
394    севастополь пьер не не пьер хотеть портфель ко...
                             ...                        
71     свой николай эпоха все герой любовь роман умол...
106    весь наташа уже описывать выглядеть роман суд ...
270    так больший что положение гореть чтобы последо...
435    перед зеландия другой эволюция что неприятно и...
102    полярный это этот мир четкость конфликт жизнь ...
Name: text, Length: 640, dtype: object

In [28]:
X_test

696    национальный быть что просто самый знание прав...
667    штат что курьезность послужить область уйма об...
63     как для наполеоновский бы во роман оливер скор...
533    этот мочь наполняться вознаграждать содержать ...
66     свой пьер нестарый отсутствие счастие реалисти...
                             ...                        
589    лондон действие тот сочетание часть находиться...
798    штат существо щеголять железо сам точка млрд б...
744    даллас или из собственный что направление мате...
513    здесь поздравлять атом все здоровый мой столь ...
670    оффенбур приниматься что он оставаться вопрос ...
Name: text, Length: 160, dtype: object

## Bag-of-Words-эмбеддинг
на всех данных обучние - чтоб все слова были учтены?

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

bof_vect = CountVectorizer()
bof_vect.fit(np.hstack([X_train, X_test]))
bof_train = bof_vect.transform(X_train)
bof_test = bof_vect.transform(X_test)

In [30]:
# пространство признаков

bof_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [31]:
bof_train.toarray().shape

(640, 23015)

## TF-IDF-Эмбеддинг

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(np.hstack([X_train, X_test]))
tfidf_train = tfidf_vect.transform(X_train)
tfidf_test = tfidf_vect.transform(X_test)

In [33]:
tfidf_train.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05868196, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [34]:
tfidf_train.toarray().shape

(640, 23015)

## Word2vec-эмбеддинг
Немного другой формат данных

In [35]:
from gensim.models import Word2Vec

X_train_w2v = X_train.apply(str.split)
X_test_w2v = X_test.apply(str.split)
w2v_vect = Word2Vec(np.hstack([X_train_w2v, X_test_w2v]), size=160, min_count=10)

In [36]:
X_train_w2v

264    [хотеть, год, роман, отражение, андрей, тот, о...
615    [естественный, небольшой, век, этот, все, разм...
329    [стиль, быть, история, осуществляться, взятка,...
342    [восторг, причем, думать, выражать, как, стано...
394    [севастополь, пьер, не, не, пьер, хотеть, порт...
                             ...                        
71     [свой, николай, эпоха, все, герой, любовь, ром...
106    [весь, наташа, уже, описывать, выглядеть, рома...
270    [так, больший, что, положение, гореть, чтобы, ...
435    [перед, зеландия, другой, эволюция, что, непри...
102    [полярный, это, этот, мир, четкость, конфликт,...
Name: text, Length: 640, dtype: object

In [37]:
w2v_vect.most_similar(positive="год")

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('земля', 0.994452178478241),
 ('если', 0.9804226160049438),
 ('более', 0.9537423849105835),
 ('вид', 0.95137619972229),
 ('до', 0.9465329647064209),
 ('мы', 0.9456946849822998),
 ('почти', 0.9454933404922485),
 ('существо', 0.9408680200576782),
 ('даже', 0.9307500720024109),
 ('теория', 0.9291499853134155)]

In [38]:
w2v_vect.most_similar(negative=["существо"])

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('наташа', 0.1432594656944275),
 ('князь', 0.1377580165863037),
 ('сказать', 0.13043634593486786),
 ('пьер', 0.126048743724823),
 ('ты', 0.12202136963605881),
 ('андрей', 0.08237704634666443),
 ('лицо', 0.07011778652667999),
 ('она', 0.06154355779290199),
 ('рука', 0.06011829525232315),
 ('ростов', 0.03334119915962219)]

In [39]:
# как представлено слово земля
w2v_vect["земля"]

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([ 3.42812926e-01,  2.74854898e-01,  7.32861757e-01, -2.26159453e-01,
        4.64013904e-01,  1.35841340e-01,  3.00892387e-02, -4.61677462e-01,
        2.99955845e-01,  4.15160090e-01,  7.80508593e-02,  3.23725998e-01,
        1.57741476e-02, -2.57165492e-01,  4.42336768e-01, -2.57141918e-01,
        2.94350851e-02, -1.69411659e-01, -5.18723130e-02,  2.66495168e-01,
        2.38380045e-01,  4.74442005e-01, -2.06381068e-01,  2.50668943e-01,
        3.78655225e-01,  4.92328942e-01,  1.03291444e-01, -6.21899106e-02,
       -5.14974952e-01,  3.63728046e-01, -1.58939362e-02,  2.39748597e-01,
        3.53962749e-01,  1.82292864e-01,  9.55085680e-02, -3.00233006e-01,
        5.50114810e-02,  1.42947957e-01, -2.81094640e-01,  4.38040316e-01,
        2.52940595e-01, -4.41990316e-01, -1.28391415e-01, -1.02178849e-01,
        1.83976829e-01,  1.18860699e-01, -3.78091305e-01,  5.18745542e-01,
        1.59270510e-01, -1.70521989e-01, -1.04986422e-01, -3.68755907e-01,
        1.74816802e-01,  

In [40]:
# усредним векторы слов
def text2vec(text):
    vecs = []
    for word in text:
        try:
            vecs.append(w2v_vect[word])
        except KeyError:
            pass
    return np.sum(vecs, axis=0) / len(vecs)

    
    
w2v_train = X_train_w2v.apply(text2vec)
w2v_test = X_test_w2v.apply(text2vec)

w2v_train

/home/alex/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


264    [0.06856898, -0.016876651, -0.07560475, 0.2633...
615    [0.1264682, 0.05371449, 0.122785605, 0.1236977...
329    [0.06822454, -0.017281411, -0.07702193, 0.2645...
342    [0.060258012, -0.026934361, -0.104194924, 0.28...
394    [0.062748395, -0.024182867, -0.09632578, 0.278...
                             ...                        
71     [0.06734159, -0.019758362, -0.084410906, 0.272...
106    [0.065937005, -0.018976681, -0.08194187, 0.265...
270    [0.06299332, -0.022997733, -0.0934749, 0.27492...
435    [0.12400533, 0.05167083, 0.11587055, 0.1252976...
102    [0.07023839, -0.014818016, -0.070251636, 0.259...
Name: text, Length: 640, dtype: object

In [41]:
w2v_train = np.dstack(w2v_train)[0]

In [42]:
w2v_train

array([[ 0.06856898,  0.1264682 ,  0.06822454, ...,  0.06299332,
         0.12400533,  0.07023839],
       [-0.01687665,  0.05371449, -0.01728141, ..., -0.02299773,
         0.05167083, -0.01481802],
       [-0.07560475,  0.12278561, -0.07702193, ..., -0.0934749 ,
         0.11587055, -0.07025164],
       ...,
       [-0.23424228, -0.32891542, -0.23371655, ..., -0.22433671,
        -0.32317835, -0.23660553],
       [ 0.0495918 ,  0.11949872,  0.04913984, ...,  0.04314175,
         0.11655879,  0.05129194],
       [-0.16969532, -0.0932226 , -0.17064446, ..., -0.17618488,
        -0.09421901, -0.1677468 ]], dtype=float32)

In [43]:
w2v_train.shape

(160, 640)

## Классификация текстов
Для каждого эмбеддинга вычислим по два суммарных вектора - для текстов истории и мира(вона и мир)

#### Для Bag-of-Words

In [44]:
# средний текст войны и мира
mir_mean_bof = np.sum( bof_train[y_train == 0], axis=0 )
mir_mean_bof.shape

(1, 23015)

In [45]:
# средний текст истории
istoria_mean_bof = np.sum( bof_train[y_train == 1], axis=0 )
istoria_mean_bof.shape

(1, 23015)

#### Для TF-IDF

In [46]:
mir_mean_tfidf = np.sum(tfidf_train[y_train == 0], axis=0 )

istoria_mean_tfidf = np.sum(tfidf_train[y_train == 1], axis=0 )


mir_mean_tfidf.shape

(1, 23015)

#### Для Word2Vec

In [47]:

mir_mean_w2v = np.sum(w2v_train[:,y_train==0], axis=1)

istoria_mean_w2v = np.sum(w2v_train[:,y_train==1], axis=1)

mir_mean_w2v.shape


(160,)

In [48]:
mir_mean_w2v

array([  21.287577 ,   -6.163924 ,  -26.476585 ,   85.83214  ,
        149.26796  ,  -56.159275 ,  101.77105  ,   35.123783 ,
         43.15454  ,   30.66188  ,   29.60446  ,  -29.40686  ,
        128.96446  , -143.71043  ,   43.52109  ,   -1.9750086,
         -7.771004 ,   28.799927 ,   35.889076 ,  -13.183698 ,
        -10.847549 ,  -18.75819  ,   16.454699 ,  -80.81402  ,
        -29.581198 ,   -0.5708198,    0.7950161,   27.714745 ,
        -50.793823 ,   41.053623 ,   48.60512  ,   81.034874 ,
        104.470184 ,   73.848885 ,    4.269104 ,   -4.525141 ,
        119.94722  ,   73.33815  ,   75.82867  ,  152.89772  ,
         81.74312  ,  -90.15021  ,  -59.52719  ,  -51.42467  ,
         38.28358  ,    4.311543 , -105.09328  ,  -36.85284  ,
         79.86898  , -125.02247  ,   29.95111  ,   66.569695 ,
        122.39261  ,   -4.8092976,   59.299915 ,   84.72803  ,
         97.293915 ,  -60.506783 ,  -57.129036 , -120.75789  ,
        -53.368374 ,   13.371088 ,   78.56778  ,   89.1

Построим датафреймы с результатами классификаций тестовых текстов. Решение - по косинусному расстоянию.

In [50]:
from scipy.spatial.distance import cosine

bof_mir = np.apply_along_axis(cosine, 1, bof_test.toarray(), v=mir_mean_bof)
bof_istoria = np.apply_along_axis(cosine, 1, bof_test.toarray(), v=istoria_mean_bof)

bof_results = pd.DataFrame ([
    bof_mir,
    bof_istoria,
    np.maximum(bof_mir, bof_istoria) != bof_istoria,
    y_test
], index=["mir", "istoria", "predict", "class"]).T.astype(np.float)

bof_results

,mir,istoria,predict,class
0,0.482883,0.311288,1.0,1.0
1,0.429743,0.289305,1.0,1.0
2,0.198570,0.286129,0.0,0.0
3,0.426814,0.311105,1.0,1.0
4,0.153879,0.312206,0.0,0.0
...,...,...,...,...
155,0.486079,0.293769,1.0,1.0
156,0.330891,0.272752,1.0,1.0
157,0.447075,0.276866,1.0,1.0
158,0.447795,0.305260,1.0,1.0


In [51]:
from sklearn.metrics import accuracy_score

accuracy_score(bof_results["predict"], bof_results["class"])

1.0

In [52]:
tfidf_mir = np.apply_along_axis(cosine, 1, tfidf_test.toarray(), v=mir_mean_tfidf)
tfidf_istoria = np.apply_along_axis(cosine, 1, tfidf_test.toarray(), v=istoria_mean_tfidf)

tfidf_results = pd.DataFrame ([
    tfidf_mir,
    tfidf_istoria,
    np.maximum(tfidf_mir, tfidf_istoria) != tfidf_istoria,
    y_test
], index=["mir", "istoria", "predict", "class"]).T.astype (np.float)

tfidf_results

,mir,istoria,predict,class
0,0.801804,0.649217,1.0,1.0
1,0.772935,0.650867,1.0,1.0
2,0.503860,0.670587,0.0,0.0
3,0.781306,0.656577,1.0,1.0
4,0.488976,0.678448,0.0,0.0
...,...,...,...,...
155,0.790895,0.634754,1.0,1.0
156,0.726309,0.614806,1.0,1.0
157,0.782911,0.639474,1.0,1.0
158,0.784943,0.664064,1.0,1.0


In [53]:
accuracy_score(tfidf_results["predict"], tfidf_results["class"])

1.0

In [54]:
# w2v

w2v_mir = np.apply_along_axis(cosine, 0, w2v_test, v=mir_mean_w2v)
w2v_istoria = np.apply_along_axis(cosine, 0, w2v_test, v=istoria_mean_w2v)

w2v_results = pd.DataFrame ([
    tfidf_mir,
    tfidf_istoria,
    np.maximum(w2v_mir, w2v_istoria) != w2v_istoria,
    y_test
], index=["mir", "istoria", "predict", "class"]).T.astype (np.float)

w2v_results



,mir,istoria,predict,class
0,0.801804,0.649217,1.0,1.0
1,0.772935,0.650867,1.0,1.0
2,0.503860,0.670587,1.0,0.0
3,0.781306,0.656577,1.0,1.0
4,0.488976,0.678448,1.0,0.0
...,...,...,...,...
155,0.790895,0.634754,1.0,1.0
156,0.726309,0.614806,0.0,1.0
157,0.782911,0.639474,0.0,1.0
158,0.784943,0.664064,1.0,1.0


In [55]:
accuracy_score(w2v_results["predict"], w2v_results["class"])

0.525

#### Попробуем применить классическую модель поверх эмбеддингов

In [56]:
from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier().fit(tfidf_train.toarray(), y_train.tolist()).score(tfidf_test.toarray(), y_test.tolist())

1.0

In [57]:
RandomForestClassifier().fit(bof_train.toarray(), y_train.tolist()).score(bof_test.toarray(), y_test.tolist())

1.0